In [1]:
import sys
import os
sys.path.append('../src/')
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import yfinance as yf
import wavy

/Users/ibiscp/GitHub/logspace/wavy/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = yf.download('BTC-USD', interval='1h', period='2y')
df = df[['Open']]

[*********************100%***********************]  1 of 1 completed


In [4]:
train = df[:int(len(df) * 0.6)]
test = df[int(len(df) * 0.4):]

In [5]:
x_train, y_train = wavy.create_panels(train, lookback=24*7, horizon=24)

In [6]:
%%time
x_train, y_train = wavy.create_panels(train, lookback=24*7, horizon=24)

x_train_last = x_train.iloc[-1]
y_train_max = y_train.max()

# TODO add index as first
y_train = y_train_max > x_train_last # if horizon max is greater than lookback max
# y_train = y_train_max > (x_train_last * 1.01)

# x here must be converted to pct change, dropna and y should match x
x_train = x_train.pct_change()
x_train = x_train.dropna()

CPU times: user 26.7 s, sys: 362 ms, total: 27 s
Wall time: 27.3 s


/Users/ibiscp/GitHub/logspace/wavy/src/wavy/panel.py:480: UserWarning: Dropped frames have different shape
  warnings.warn("Dropped frames have different shape")


In [21]:
y_train[0]

,Open
0,1


In [7]:
y_train.as_dataframe().value_counts()

Open 
True     9129
False     976
dtype: int64

# Train model

In [8]:
conv = wavy.models.ConvModel(x_train, y_train, 'classification')
conv.fit(epochs=100, verbose=True)
conv.score()

Epoch 1/100


/Users/ibiscp/GitHub/logspace/wavy/src/wavy/models.py:525: UserWarning: Kernel size is not a divisor of lookback.
  warnings.warn("Kernel size is not a divisor of lookback.")
2022-06-21 23:37:46.582718: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


228/228 [==============================] - 2s 5ms/step - loss: 0.3203 - auc: 0.5064 - accuracy: 0.9094 - val_loss: 0.4276 - val_auc: 0.5433 - val_accuracy: 0.8670
Epoch 2/100
228/228 [==============================] - 1s 3ms/step - loss: 0.3056 - auc: 0.5042 - accuracy: 0.9102 - val_loss: 0.4097 - val_auc: 0.5371 - val_accuracy: 0.8670
Epoch 3/100
228/228 [==============================] - 1s 3ms/step - loss: 0.3058 - auc: 0.5135 - accuracy: 0.9102 - val_loss: 0.3985 - val_auc: 0.5788 - val_accuracy: 0.8670
Epoch 4/100
228/228 [==============================] - 1s 3ms/step - loss: 0.3053 - auc: 0.5053 - accuracy: 0.9102 - val_loss: 0.4173 - val_auc: 0.5934 - val_accuracy: 0.8670
Epoch 5/100
228/228 [==============================] - 1s 3ms/step - loss: 0.3015 - auc: 0.5431 - accuracy: 0.9102 - val_loss: 0.3887 - val_auc: 0.5963 - val_accuracy: 0.8670
Epoch 6/100
228/228 [==============================] - 1s 4ms/step - loss: 0.3004 - auc: 0.5541 - accuracy: 0.9102 - val_loss: 0.3884 - v

,train,test,val
loss,0.008499,0.896655,1.539288
auc,1.000000,0.509266,0.505694
accuracy,0.999725,0.887240,0.844970


In [22]:
# Test
pred = conv.predict()

In [24]:
# TODO Fix index for pred
# TODO add graph showing x and vertical lines when y = 1
pred.plot()

In [9]:
labels = y_train.test.as_dataframe()

In [10]:
predictions = conv.predict().test.as_dataframe()

In [11]:
results = pd.DataFrame()
results['Labels'] = labels.Open
results['Predicted'] = predictions.Open
results['Predicted Probability'] = conv.predict_proba().test.as_dataframe().Open
results['Error'] = conv.residuals().test.as_dataframe().Open
results['Error Probability'] = predictions.Open - conv.predict_proba().test.as_dataframe().Open
results.reset_index(drop=True, inplace=True)

In [12]:
results

,Labels,Predicted,Predicted Probability,Error,Error Probability
0,1,1,1.000000,0,0.000000e+00
1,1,1,0.999990,0,9.536743e-06
2,1,1,1.000000,0,0.000000e+00
3,1,1,1.000000,0,0.000000e+00
4,1,1,1.000000,0,0.000000e+00
...,...,...,...,...,...
1006,1,1,1.000000,0,4.768372e-07
1007,1,1,1.000000,0,0.000000e+00
1008,1,1,0.999946,0,5.435944e-05
1009,1,1,0.999999,0,6.556511e-07


In [13]:
true_positives = results[(results['Predicted'] == 1) & (results['Labels'] == 1)].shape[0]
false_positives = results[(results['Predicted'] == 1) & (results['Labels'] == 0)].shape[0]
true_negatives = results[(results['Predicted'] == 0) & (results['Labels'] == 0)].shape[0]
false_negatives = results[(results['Predicted'] == 0) & (results['Labels'] == 1)].shape[0]

print(f"True Positives: {true_positives}/{results.shape[0]}")
print(f"False Positives: {false_positives}/{results.shape[0]}")
print(f"True Negatives: {true_negatives}/{results.shape[0]}")
print(f"False Negatives: {false_negatives}/{results.shape[0]}")

True Positives: 660/1011
False Positives: 53/1011
True Negatives: 28/1011
False Negatives: 270/1011


In [14]:
conv.get_roc()

0.9987417

In [15]:
results['Predicted 0.8'] = results['Predicted Probability'] > conv.get_roc() + 0.05

true_positives = results[(results['Predicted 0.8'] == 1) & (results['Labels'] == 1)].shape[0]
false_positives = results[(results['Predicted 0.8'] == 1) & (results['Labels'] == 0)].shape[0]
true_negatives = results[(results['Predicted 0.8'] == 0) & (results['Labels'] == 0)].shape[0]
false_negatives = results[(results['Predicted 0.8'] == 0) & (results['Labels'] == 1)].shape[0]

print(f"True Positives: {true_positives}/{results.shape[0]}")
print(f"False Positives: {false_positives}/{results.shape[0]}")
print(f"True Negatives: {true_negatives}/{results.shape[0]}")
print(f"False Negatives: {false_negatives}/{results.shape[0]}")

True Positives: 0/1011
False Positives: 0/1011
True Negatives: 81/1011
False Negatives: 930/1011


In [16]:
results['Error Probability'].plot()

# Test buying and selling

In [17]:
%%time
x_test, _ = wavy.create_panels(test, lookback=24*7, horizon=24)

# x here must be converted to pct change, dropna and y should match x
x_test_pct = x_test.copy()
x_test_pct = x_test_pct.pct_change()
x_test_pct = x_test_pct.dropna()

CPU times: user 20.8 s, sys: 521 ms, total: 21.3 s
Wall time: 21.8 s


/Users/ibiscp/GitHub/logspace/wavy/src/wavy/panel.py:480: UserWarning:

Dropped frames have different shape



In [18]:
predicted = conv.predict(x_test_pct)

In [19]:
VALUE = 10000
QUANTITY = 0

for x, y in zip(x_test, predicted):
    if y.iloc[0]['Open'] == 1 and VALUE > 0:
        QUANTITY = VALUE/x.iloc[-1]['Open']
        VALUE = 0
        print(f"Buy ({str(x.index[-1].date())}) - Price: {x.iloc[-1]['Open']} Quantity: {QUANTITY}")
    elif y.iloc[0]['Open'] == 0 and QUANTITY > 0:
        VALUE = QUANTITY * x.iloc[-1]['Open']
        QUANTITY = 0
        print(f"Sell ({str(x.index[-1].date())}) - Price: {x.iloc[-1]['Open']} Value: {VALUE}")    

Buy (2021-04-17) - Price: 61116.30859375 Quantity: 0.16362244759367944
Sell (2021-04-17) - Price: 61187.9375 Value: 10011.720096959083
Buy (2021-04-17) - Price: 60889.3359375 Quantity: 0.16442485277283425
Sell (2021-04-17) - Price: 60722.5625 Value: 9984.298399051726
Buy (2021-04-17) - Price: 60929.90625 Quantity: 0.16386531694444756
Sell (2021-04-17) - Price: 60373.65625 Value: 9893.148316501378
Buy (2021-04-17) - Price: 60991.734375 Quantity: 0.16220473836134255
Sell (2021-04-17) - Price: 61401.96484375 Value: 9959.689642352823
Buy (2021-04-18) - Price: 56807.8046875 Quantity: 0.17532255817912912
Sell (2021-04-18) - Price: 57588.4375 Value: 10096.55218403889
Buy (2021-04-18) - Price: 55006.67578125 Quantity: 0.1835513970011705
Sell (2021-04-19) - Price: 57361.42578125 Value: 10528.769836127396
Buy (2021-04-19) - Price: 56824.95703125 Quantity: 0.18528425512644495
Sell (2021-04-19) - Price: 57126.9453125 Value: 10584.723509875717
Buy (2021-04-19) - Price: 56561.19921875 Quantity: 0.18

# Other models

In [ ]:
dense = wavy.models.DenseModel(x, ylabels, 'classification')
dense.fit(epochs=100, verbose=True)
dense.score()

In [ ]:
conv = wavy.models.ConvModel(x, ylabels, 'classification')
conv.fit(epochs=100, verbose=True)
conv.score()

In [ ]:
residuals = conv.residuals()

In [ ]:
len(residuals)

In [ ]:
residuals.sample(10000, how='spaced').plot()